In [1]:
from dataloader import load_data
import torch
import torch.nn as nn
import utils

### Let's open the training and validation files containing examples for top quarks (signal) and QCD jets (background)

In [2]:
input_folder = '/pscratch/sd/v/vmikuni/datasets'
train_data = load_data('top',input_folder,batch=128,dataset_type='train',num_evt = 100_000)
val_data = load_data('top',input_folder,batch=128,dataset_type='val')

In [3]:
print (f"Loading {len(train_data)} batches of events for training and {len(val_data)} for validation")

Loading 781 batches of events for training and 3148 for validation


### We Now need to create a model that will take the data as input and predict a label for each data entry. Let's create a config file with the network parameters

In [4]:
config = {
    'num_layers': 2,
    'hidden_dim': 64,
    'activation': nn.ReLU(), #https://docs.pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
}

In [5]:
class SimpleDeepSets(nn.Module):
    def __init__(self, input_dim, config, num_classes=2):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, config["hidden_dim"])
        
        layers = []
        for _ in range(config["num_layers"]):
            layers.append(nn.Linear(config["hidden_dim"], config["hidden_dim"]))
            layers.append(config["activation"])
        self.hidden_layers = nn.ModuleList(layers)

        self.output_layer = nn.Linear(config["hidden_dim"], num_classes)

    def forward(self, inputs):
        zero_pad_mask = (inputs[:, :, 2] != 0).unsqueeze(-1).float()
        x = self.input_layer(inputs) * zero_pad_mask
        for layer in self.hidden_layers:
            x = layer(x) * zero_pad_mask
        x = x.mean(1)  # aggregate over particles
        return self.output_layer(x)

In [6]:
model = SimpleDeepSets(input_dim=4,config=config) #remember the inputs are delta eta, delta phi, log(pT), log(E)

### Now we are going to create the training class that will train the model, but first, let's set up the learning rate and the optimizer

In [7]:
optimizer = torch.optim.Adam
lr = 5e-4
epochs = 100
patience = 10 # Number of consecutive epochs to stop the training if the validation loss does not improve

In [8]:
trainer = utils.Trainer(train_data,val_data,model,lr,optimizer)

### Let's train the model!

In [9]:
trainer.train(epochs)

Epoch 1: train loss=0.5546, validation loss=0.4841
Epoch 2: train loss=0.3944, validation loss=0.3645
Epoch 3: train loss=0.3616, validation loss=0.3529
Epoch 4: train loss=0.3532, validation loss=0.3482
Epoch 5: train loss=0.3485, validation loss=0.3544
Epoch 6: train loss=0.3455, validation loss=0.3408
Epoch 7: train loss=0.3436, validation loss=0.3409
Epoch 8: train loss=0.3410, validation loss=0.3379
Epoch 9: train loss=0.3415, validation loss=0.3392
Epoch 10: train loss=0.3400, validation loss=0.3364
Epoch 11: train loss=0.3393, validation loss=0.3373
Epoch 12: train loss=0.3402, validation loss=0.3535
Epoch 13: train loss=0.3381, validation loss=0.3397
Epoch 14: train loss=0.3384, validation loss=0.3496
Epoch 15: train loss=0.3371, validation loss=0.3377
Epoch 16: train loss=0.3372, validation loss=0.3337
Epoch 17: train loss=0.3359, validation loss=0.3353
Epoch 18: train loss=0.3357, validation loss=0.3457
Epoch 19: train loss=0.3351, validation loss=0.3414
Epoch 20: train loss=

### Now let's evaluate the model

In [10]:
test_data = load_data('top',input_folder,batch=128,dataset_type='test')
predictions, labels = trainer.evaluate(test_data)

In [11]:
utils.print_metrics(predictions,labels)

AUC: 0.9243

ACC: 0.8906

Signal class 1 vs Background class 0:
Class 1 effS at 0.30004107507831485 1.0/effB = 21.101484113712374
Class 1 effS at 0.5000668088623194 1.0/effB = 15.395684001830105


### Try changing the hyperparameters, activation functions, layers, learning rate!